In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [9]:
def generate_time_series(batch_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10)) # wave 1
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20)) # wave 2
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5) # + noise
    return series[..., np.newaxis].astype(np.float32)

In [10]:
n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

In [12]:
y_pred = X_valid[:, -1]

In [13]:
y_pred

array([[ 0.51692134],
       [-0.4624161 ],
       [ 0.11211568],
       ...,
       [ 0.5166377 ],
       [ 0.18039198],
       [-0.14733098]], dtype=float32)

In [14]:
np.mean(keras.losses.mean_squared_error(y_valid, y_pred))

0.020760437

In [15]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[50, 1]),
    keras.layers.Dense(1)
])

In [19]:
model.compile(
    loss='mean_squared_error',
    optimizer='Adam'
)

In [20]:
model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

Train on 7000 samples, validate on 1000 samples
Epoch 1/20
7000/7000 [==============================] - 1s 149us/sample - loss: 0.0038 - val_loss: 0.0036
Epoch 2/20
7000/7000 [==============================] - 0s 70us/sample - loss: 0.0037 - val_loss: 0.0034
Epoch 3/20
7000/7000 [==============================] - 0s 71us/sample - loss: 0.0036 - val_loss: 0.0033
Epoch 4/20
7000/7000 [==============================] - 1s 90us/sample - loss: 0.0035 - val_loss: 0.0032
Epoch 5/20
7000/7000 [==============================] - 0s 70us/sample - loss: 0.0035 - val_loss: 0.0032
Epoch 6/20
7000/7000 [==============================] - 1s 76us/sample - loss: 0.0035 - val_loss: 0.0032
Epoch 7/20
7000/7000 [==============================] - 0s 71us/sample - loss: 0.0034 - val_loss: 0.0032
Epoch 8/20
7000/7000 [==============================] - 1s 80us/sample - loss: 0.0034 - val_loss: 0.0031
Epoch 9/20
7000/7000 [==============================] - 0s 70us/sample - loss: 0.0034 - val_loss: 0.0030
Epoch 

In [21]:
model.evaluate(X_valid, y_valid)

2000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

0.00317015047930181

In [24]:
model_rnn = keras.models.Sequential([
    keras.layers.SimpleRNN(1, input_shape=[None, 1])
])

In [25]:
model_rnn.compile(
    loss='mean_squared_error',
    optimizer='Adam'
)

In [26]:
model_rnn.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

Train on 7000 samples, validate on 1000 samples
Epoch 1/20
7000/7000 [==============================] - 6s 789us/sample - loss: 0.3050 - val_loss: 0.2874
Epoch 2/20
7000/7000 [==============================] - 3s 494us/sample - loss: 0.2612 - val_loss: 0.2432
Epoch 3/20
7000/7000 [==============================] - 3s 483us/sample - loss: 0.2217 - val_loss: 0.2039
Epoch 4/20
7000/7000 [==============================] - 4s 504us/sample - loss: 0.1874 - val_loss: 0.1713
Epoch 5/20
7000/7000 [==============================] - 4s 514us/sample - loss: 0.1597 - val_loss: 0.1449
Epoch 6/20
7000/7000 [==============================] - 4s 534us/sample - loss: 0.1379 - val_loss: 0.1238
Epoch 7/20
7000/7000 [==============================] - 4s 532us/sample - loss: 0.1183 - val_loss: 0.1035
Epoch 8/20
7000/7000 [==============================] - 4s 521us/sample - loss: 0.0968 - val_loss: 0.0840
Epoch 9/20
7000/7000 [==============================] - 4s 542us/sample - loss: 0.0778 - val_loss: 0.067

In [27]:
model_rnn.evaluate(X_valid, y_valid)

2000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

0.011609072148799896

In [29]:
model_deep_rnn = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.SimpleRNN(1)
])

In [30]:
model_deep_rnn.compile(
        optimizer='Adam', loss='mean_squared_error'
)

In [31]:
model_deep_rnn.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

Train on 7000 samples, validate on 1000 samples
Epoch 1/20
7000/7000 [==============================] - 17s 2ms/sample - loss: 0.0543 - val_loss: 0.0077
Epoch 2/20
7000/7000 [==============================] - 12s 2ms/sample - loss: 0.0062 - val_loss: 0.0053
Epoch 3/20
7000/7000 [==============================] - 12s 2ms/sample - loss: 0.0047 - val_loss: 0.0042
Epoch 4/20
7000/7000 [==============================] - 12s 2ms/sample - loss: 0.0042 - val_loss: 0.0041
Epoch 5/20
7000/7000 [==============================] - 12s 2ms/sample - loss: 0.0039 - val_loss: 0.0038
Epoch 6/20
7000/7000 [==============================] - 12s 2ms/sample - loss: 0.0036 - val_loss: 0.0036
Epoch 7/20
7000/7000 [==============================] - 12s 2ms/sample - loss: 0.0036 - val_loss: 0.0033
Epoch 8/20
7000/7000 [==============================] - 12s 2ms/sample - loss: 0.0034 - val_loss: 0.0044
Epoch 9/20
7000/7000 [==============================] - 13s 2ms/sample - loss: 0.0034 - val_loss: 0.0032
Epoch 1

In [34]:
series_ahead = generate_time_series(1, n_steps + 10)
X_new, y_new = series[:, :n_steps], series[:, n_steps:]
X = X_new

for step_ahead in range(10):
    y_pred_one = model.predict(X[:, step_ahead:])[:, np.newaxis, :]
    X = np.concatenate([X, y_pred_one], axis=1)
    
Y_pred = X[:, n_steps:]

In [40]:
series_all = generate_time_series(10000, n_steps + 10)
X_train, Y_train = series[:7000, :n_steps], series[:7000, -10:, 0]
X_valid, Y_valid = series[7000:9000, :n_steps], series[7000:9000, -10:, 0]
X_test, Y_test = series[9000:, :n_steps], series[9000:, -10:, 0]

In [41]:
model_all = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(10)
])

In [42]:
model_all.compile(optimizer='Adam', loss='mean_squared_error')

In [43]:
model_all.fit(X_train, Y_train, epochs=20, validation_data=(X_test, Y_test))

Train on 7000 samples, validate on 1000 samples
Epoch 1/20
7000/7000 [==============================] - 11s 2ms/sample - loss: 0.0324 - val_loss: 0.0091
Epoch 2/20
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0053 - val_loss: 0.0031
Epoch 3/20
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0022 - val_loss: 0.0017
Epoch 4/20
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0014 - val_loss: 0.0012
Epoch 5/20
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0011 - val_loss: 0.0010
Epoch 6/20
7000/7000 [==============================] - 9s 1ms/sample - loss: 9.5412e-04 - val_loss: 9.2476e-04
Epoch 7/20
7000/7000 [==============================] - 9s 1ms/sample - loss: 8.5467e-04 - val_loss: 8.1210e-04
Epoch 8/20
7000/7000 [==============================] - 8s 1ms/sample - loss: 7.8934e-04 - val_loss: 7.6678e-04
Epoch 9/20
7000/7000 [==============================] - 8s 1ms/sample - loss: 7.1906e-04 - val_

In [44]:
Y_pred = model_all.predict(X_new)

In [45]:
Y_pred

array([[ 0.39761922,  0.20087613, -0.16235201, ..., -0.28944322,
        -0.30629918, -0.25226158],
       [-0.18506308, -0.17205049, -0.2644793 , ..., -0.38328347,
        -0.21753699, -0.03669877],
       [ 0.6147266 ,  0.5049024 ,  0.16565198, ..., -0.11848017,
        -0.206887  , -0.32685372],
       ...,
       [ 0.2976464 ,  0.3283936 ,  0.40769184, ..., -0.08610561,
        -0.32982934, -0.5536278 ],
       [ 0.26556778,  0.3304501 ,  0.45936906, ...,  0.16468929,
        -0.09034996, -0.3045127 ],
       [-0.32928836, -0.3551129 , -0.37160766, ..., -0.37397268,
        -0.29383177, -0.19076283]], dtype=float32)

In [46]:
model_all.evaluate(X_valid, Y_valid)

2000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

0.00044121386646293106